In [1]:
# ==============================================================================
# Importing Library
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.document_loaders import Docx2txtLoader
import threading
from langchain_core.pydantic_v1 import BaseModel, Field, validator, root_validator
from langchain_community.document_loaders import FireCrawlLoader
from langchain_groq import ChatGroq
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from typing import List
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
# from src.models import ResponseModel, SentenceParser
from langchain.chains import LLMChain
import concurrent.futures
from difflib import ndiff
import pandas as pd

In [2]:
os.environ["GROQ_API_KEY"] = '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'
open_ai_key = 'sk-None-YYN2MievH2m1islKXEIkT3BlbkFJ9vmzUu8bHOOXgK9498jA'
os.environ["OPENAI_API_KEY"] = open_ai_key

In [3]:
# ==============================================================================
from typing import List, Optional
from pydantic import BaseModel, Field, root_validator, ValidationError

class SentenceParser(BaseModel):
    originalContent: str = Field(..., description="Original Sentence with no correction")
    correctedSentence: str = Field(..., description="Corrected Sentence in English UK. Enclosed corrections in <b></b>.")
    

    @root_validator(pre=True)
    def fill_missing_fields(cls, values):
        values['correctedSentence'] = values.get('correctedSentence', '')
        return values

# Define ResponseModel to include sentences
class ResponseModel(BaseModel):
    sentences: List[SentenceParser]

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2335332208.py:10: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @root_validator(pre=True)


In [4]:
def get_html(url,mode='scrape',ignore_pages=[],max_pages=4):
    """
    Retrieves HTML content from a given URL.
    Args:
        url (str): The URL from which to retrieve the HTML content.
        mode (str, optional): The mode in which to retrieve the HTML content. 
            Defaults to 'scrape'. Options are 'scrape' or 'crawl'.
        ignore_pages (List[str], optional): A list of pages to ignore when 
            retrieving the HTML content. Defaults to an empty list.
        max_pages (int, optional): The maximum number of pages to retrieve 
            when in 'crawl' mode. Defaults to 4.
    Returns:
        List[Document]: A list of Document objects representing the HTML content 
            retrieved from the URL.

    Raises:
        None

    """
    if mode == 'scrape':
        loader = FireCrawlLoader(
        api_key="fc-d22f7f0d1f7c46949c24b237959797f6", url=url, mode=mode
            )
    else:
        crawl_params = {
                'crawlerOptions': {
                    'excludes': [
                        'blog/*', 'login/*', 'account/*', 'user/*', 'profile/*',
                        'admin/*', 'dashboard/*', 'search/*', 'filter/*',
                        'checkout/*', 'payment/*', 'cart/*', 'css/*', 'js/*',
                        'images/*', 'assets/*', 'temp/*', 'under-construction/*',
                        'api/*', '404', '500', 'downloads/*', 'files/*', 'pdfs/*',
                        'archive/*', 'old/*', 'version/*', 'forum/*', 'comments/*',
                        'reviews/*', 'external/*', 'outbound/*', 'product/*', 'shop/*',
                        'category/*', 'promo/*', 'deals/*', 'offers/*', 'help/*',
                        'support/*', 'news/*', 'press/*', 'events/*',
                        'calendar/*', 'subscribe/*', 'signup/*', 'test/*', 'staging/*',
                        'wp-admin/*', 'backend/*', 'admin-panel/*', 'management/*'
                    ],
                    'includes': [], # leave empty for all pages
                    'limit': max_pages,
                }
            }

        loader = FireCrawlLoader(
        api_key="fc-d22f7f0d1f7c46949c24b237959797f6", url=url, mode=mode,
        params=crawl_params
            ) 
        
    # crawl or scrape the website
    docs = loader.load()
    
    return docs

def remove_all_repetitions(text):

    """
    Removes all repetitions of words in a given text.

    Args:
        text (str): The input text from which repetitions will be removed.

    Returns:
        str: The text with all repetitions removed, where each word appears only once.
    """

    words = text.split()
    seen = set()
    result = []

    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)

    return ' '.join(result)

def extract_plain_text_from_markdown(markdown_content):
    """
    Extracts plain text from Markdown content by removing image tags, links, HTML tags, and symbols.

    Args:
        markdown_content (str): The Markdown content to extract plain text from.

    Returns:
        str: The plain text extracted from the Markdown content.

    Removes image tags (Markdown and HTML) by replacing the image tag with an empty string.
    Removes links (Markdown and plain URLs) by replacing the link with an empty string.
    Removes any remaining inline links by replacing the inline link with an empty string.
    Removes HTML tags (including SVG) by replacing the HTML tag with an empty string.
    Removes all symbols except ., "", '', ?, and , by replacing them with an empty string.
    Removes any extra whitespace and newlines by replacing multiple whitespace characters with a single space and stripping leading and trailing whitespace.
    """
    # Remove image tags (Markdown and HTML)
    markdown_content = re.sub(r'!\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'<img.*?>', '', markdown_content)
    
    # Remove links (Markdown and plain URLs)
    markdown_content = re.sub(r'\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'\(https?://.*?\)', '', markdown_content)
    markdown_content = re.sub(r'https?://\S+|www\.\S+', '', markdown_content)
    
    # Remove any remaining inline links
    markdown_content = re.sub(r'\[.*?\]', '', markdown_content)
    
    # Remove HTML tags (including SVG)
    markdown_content = re.sub(r'<.*?>', '', markdown_content)
    
    # Remove all symbols except ., "", '', ?, and ,
    markdown_content = re.sub(r"[^a-zA-Z0-9\s.,\"'?']", '', markdown_content)
    
    # Remove any extra whitespace and newlines
    markdown_content = re.sub(r'\s+', ' ', markdown_content).strip()
    
    return markdown_content

In [5]:
def get_splited_text(text):
    """
    Splits the given text into multiple documents using the RecursiveCharacterTextSplitter.

    Args:
        text (str): The input text to be split.

    Returns:
        List[Document]: A list of Document objects representing the split text.

    This function takes a text as input and splits it into multiple documents using the
    RecursiveCharacterTextSplitter. The chunk size is set to 8000, with a chunk overlap of
    100. The length function used is the built-in len function. The is_separator_regex
    parameter is set to False. The input text is passed as a list to the create_documents
    method of the text_splitter object. The resulting documents are returned as a list.

    Example usage:
        text = "This is a sample text to be split."
        documents = get_splited_text(text)
        print(documents)

    Output:
        [Document(page_content='This is a sample text to be split.', metadata={}), ...]
    """
    text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=8000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
    )
    texts = text_splitter.create_documents([text])
    return texts

In [6]:
from urllib.parse import urlparse

def get_base_url(url):
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    return base_url
    

In [9]:
from src.db_driver import *

/Users/ali/anaconda3/envs/llm_langchain/lib/python3.10/site-packages/pymongo/ocsp_support.py:280: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to this_update_utc.
  if response.this_update > now:
/Users/ali/anaconda3/envs/llm_langchain/lib/python3.10/site-packages/pymongo/ocsp_support.py:284: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if response.next_update and response.next_update < now:
/Users/ali/anaconda3/envs/llm_langchain/lib/python3.10/site-packages/pymongo/ocsp_cache.py:65: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if value.next_update is None:
/Users/ali/anaconda3/envs/llm_langchain/lib/python3.10/site-packages/pymongo/ocsp_cache.py:71: CryptographyDeprecationWarning: Properties that return a naïve datetime object have be

Pinged your deployment. You successfully connected to MongoDB!


In [59]:
from datetime import datetime


def reterive_split_text(url,mode,ignore_pages=[],max_pages=4,enable_db=False):
    """
    Retrieves HTML content from a given URL and splits it into multiple documents.

    Args:
        url (str): The URL from which to retrieve the HTML content.
        mode (str): The mode of retrieval, either 'crawl' or 'scrape'.
        ignore_pages (List[int]): A list of page numbers to ignore.
        max_pages (int): The maximum number of pages to retrieve.

    Returns:
        List[Document]: A list of Document objects representing the split text.

    """
    documents_list = []
    html_content = get_html(url,mode=mode,ignore_pages=ignore_pages,max_pages=max_pages)
    print('Crawled')
    base_page_url = get_base_url(url)
    if enable_db:
        
        status = add_parrent_doc(base_page_url)
        if status:
            print('Website added!',status)
    for document in html_content:
        document_dict = {}
        chunk_list = []
        plain_text = extract_plain_text_from_markdown(document.page_content)
        plain_text = remove_all_repetitions(plain_text)
        metadata = document.dict().get('metadata',{})
        url = metadata.get('ogUrl','')
        title = metadata.get('ogTitle','')
        document_dict = {'url':url,'title':title,'page_content':plain_text}
        chunks = get_splited_text(plain_text)
        for chunk in chunks:
            chunk_list.append(chunk.page_content)
        document_dict['chunks'] = chunk_list
        document_dict['status'] = 'init'
        document_dict['timestamp'] = datetime.now()
        document_dict['parent_url'] = base_page_url
        documents_list.append(document_dict)

    # print(documents_list)
    if enable_db:
        status_1 = add_processed_pages(documents_list)
        print(status_1)
        

    return documents_list

In [11]:
document_list = reterive_split_text(url="https://www.smilecliniq.com/",mode="crawl",max_pages=3,enable_db=True)

Website added! UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000034'), 'opTime': {'ts': Timestamp(1723108591, 1), 't': 52}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1723108591, 1), 'signature': {'hash': b'o\x1e\x95\xef}J\x94\tB\xd5\x87\x08>"`X\xb9L,8', 'keyId': 7353692852154531846}}, 'operationTime': Timestamp(1723108591, 1), 'updatedExisting': True}, acknowledged=True)
InsertManyResult([ObjectId('66b48cee940cc27674a58747'), ObjectId('66b48cee940cc27674a58748'), ObjectId('66b48cee940cc27674a58749')], acknowledged=True)


In [12]:
abc = read_processed_pages({})

In [76]:
def get_first_two_init_pages_with_parent(parent_doc_url):
    # Find the parent document
    parent_doc = parent_docs_collection.find_one({'url': parent_doc_url})
    
    if not parent_doc:
        return None, f"No parent document found with URL: {parent_doc_url}"
    
    
    # Find the first two documents with status 'init' and matching parent_url
    query = {'status': 'init', 'parent_url': parent_doc_url}
    documents = processed_pages_collection.find(query).limit(2)
    
    return [doc for doc in documents]

In [50]:
reterived_documents = get_first_two_init_pages_with_parent('https://www.smilecliniq.com')


In [51]:
reterived_documents

[]

In [20]:
def get_corrected_content(page_content):

    """
    Retrieves corrected content from a given page content.

    Args:
        page_content (str): The content of the page to be corrected.

    Returns:
        Union[str, dict]: The corrected content as a JSON object with a single key "sentences"
        which contains a list of objects. Each object should have keys "originalContent" and
        "correctedSentence". If the result is not a dictionary, it is returned as is.

    Raises:
        None

    """

    llm = ChatGroq(
            model="llama-3.1-70b-versatile",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=3,
            # other params...
        )
    

    parser = PydanticOutputParser(pydantic_object=ResponseModel)

    prompt = PromptTemplate(
        template="""
        You are a smart assistant.
        You will be given a page-long document.
        You will return a list of sentences in the page.
        If there is any HTML content or non-textual content, you will remove it.
        First, you will clean the text by removing the images and links, and extract plain text.
        You will not expand any point or enhance anything. 
        If sentence contain the social media or web development related things like cookies etc you will remove it.
        Just return what is in the original text.
        Provide the English UK version of sentences, performing both spelling and grammar checks.
        Return the result as a JSON object with a single key "sentences" which contains a list of objects.
        Each object should have keys "originalContent" and "correctedSentence".

        \n\n{query}\n
        """,
        input_variables=["query"]
    )

    chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)

    result = chain({"query": page_content})

    if isinstance(result,dict):
        result = result['text']
        return result
    return result

In [22]:
def process_document(doc):
    """
    Corrects the grammar of text chunks in a single document using ThreadPoolExecutor.

    Args:
        doc (dict): A dictionary representing the document with 'url', 'title',
                    'page_content', and 'chunks'.

    Returns:
        dict: The updated document with corrected text chunks.
    """
    with concurrent.futures.ThreadPoolExecutor() as chunk_executor:
        chunks = doc['chunks']
        chunk_futures = {chunk_executor.submit(get_corrected_content, chunk): chunk for chunk in chunks}
        corrected_chunks = [future.result() for future in concurrent.futures.as_completed(chunk_futures)]
        doc['chunks_corrected'] = corrected_chunks
    return doc

In [24]:
def grammar_check_main_concurrent(doc_list=[]):
    """
    Corrects the grammar of text chunks in each document using ThreadPoolExecutor and yields each processed document.

    Args:
        doc_list (list): List of documents where each document is a dictionary
                         with 'url', 'title', 'page_content', and 'chunks'.

    Yields:
        dict: The updated document with corrected text chunks.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        doc_futures = {executor.submit(process_document, doc): doc for doc in doc_list}
        for future in concurrent.futures.as_completed(doc_futures):
            corrected_doc = future.result()
            yield corrected_doc
            # Process the document further as needed here (e.g., store in database, display)
            print(f"Processed document: {corrected_doc['title']}")


def highlight_changes(original, corrected):
    """
    Generates a highlighted version of the changes between the original and corrected strings.

    Args:
        original (str): The original string.
        corrected (str): The corrected string.

    Returns:
        str: The highlighted version of the changes.

    Example:
        >>> original = "Hello world!"
        >>> corrected = "Hello World!"
        >>> highlight_changes(original, corrected)
        '<b>H</b>ello <b>W</b>orld!'
    """
    diff = list(ndiff(original.split(), corrected.split()))
    result = []
    in_change = False

    for d in diff:
        if d.startswith('- '):
            continue
        elif d.startswith('+ '):
            if not in_change:
                result.append('<b>')
                in_change = True
            result.append(d[2:])
        elif d.startswith('  '):
            if in_change:
                result.append('</b>')
                in_change = False
            result.append(d[2:])
    
    if in_change:
        result.append('</b>')

    return ' '.join(result)


# Function to add <b></b> around caution words
def highlight_caution_words(sentence, caution_word_list):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(word) for word in caution_word_list) + r')\b', re.IGNORECASE)
    highlighted_sentence = pattern.sub(r'<b>\1</b>', sentence)
    if highlighted_sentence != sentence:
        return highlighted_sentence
    return None

def normalize_and_filter(df, column_name):
    # Create a temporary column in lowercase for normalization and matching
    df['temp_lower'] = df[column_name].str.lower()
    
    # Pattern to match social media names, specific extensions, and keywords like 'cookies'
    pattern = r'\b(company|facebook|twitter|instagram|linkedin|youtube|\.com|\.ad|\.gov|\.uk|\.php|\.asp|cookies|cookies|statistical|tracking|ads|script|event|policy|Cloudflare|cookie|cfduid|llc|party)\b'
    
    # Filter rows where the temporary lowercase column does not contain any of the specified patterns
    filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]
    
    # Drop the temporary column
    filtered_df.drop('temp_lower', axis=1, inplace=True)
    
    return filtered_df

def get_corrected_highlights(temp_doc,caution_word_list=["Best", "Specialist", "Specialised", "Finest",
                                                          "Most experienced",
                                "Superior", "Principle", "Expert", "Amazing", "Speciality",
                                "leader", "leaders", "service", "implantologist"
            ]):
    temp_sent_list = []
    for doc in temp_doc:
        for sentence in doc.dict().get('sentences',[]):
            temp_ab = highlight_changes(sentence.get('originalContent',[]),sentence.get('correctedSentence',[]))
            orignal_content = sentence.get('originalContent','')
            cc_content = sentence.get('correctedSentence','')
            highlighted = temp_ab
            highlight_sentence = {}
            highlight_sentence['originalContent'] = orignal_content
            highlight_sentence['correctedSentence'] = cc_content
            highlight_sentence['highlighted'] = highlighted
            temp_sent_list.append(highlight_sentence)


    df = pd.DataFrame(temp_sent_list)

    filtered_df = df[df['highlighted'].str.contains('<b>')]
    filtered_df = normalize_and_filter(filtered_df,'highlighted')
    org_sentences = df['originalContent'].tolist()
    highlighted_sentences = [highlight_caution_words(sentence,caution_word_list=caution_word_list) for sentence in org_sentences if highlight_caution_words(sentence,caution_word_list)]
    flatten_doc_list = filtered_df.to_dict('records')

    ret_docs = {}
    ret_docs['table_data'] = flatten_doc_list
    ret_docs['caution_sentences'] = highlighted_sentences

    return ret_docs




In [28]:
def update_processed_documents(documents):
    for doc in documents:
        # Extract the ID from the document
        document_id = doc['_id']
        
        # Create the filter criteria using the document ID
        filter_criteria = {'_id': document_id}
        
        # Create the update operation (remove _id from the update values)
        update_values = {key: value for key, value in doc.items() if key != '_id'}
        update_operation = {'$set': update_values}
        
        
        # Perform the update_one operation
        result = processed_pages_collection.update_one(filter_criteria, update_operation)
        
        if result.matched_count == 0:
            print(f"No document found with _id: {document_id} to update.")
        else:
            print(f"Document with _id: {document_id} updated successfully.")
            
    return "Documents updated successfully."

In [27]:
reterived_documents

[{'_id': ObjectId('66b48ab7f8c0761c30ba8d1d'),
  'url': 'https://www.smilecliniq.com/medical-aesthetics.php',
  'title': 'Medical Aesthetics',
  'page_content': 'OPEN EVENINGS AND WEEKENDS St John\'s Wood Finchley Dentist Close Dental Care Orthodontics Implants Specialist Services Laser Dentistry Other Medical Aesthetics Previous "My wife needed to see a dentist on an emergency basis Saturday after bad fall the evening before. We called morning and they were able take us that afternoon. The quality of care, both technically interpersonally, could not have been better. If we lived..." Hal N. "I\'ve going Smile Cliniq for past couple years, treatment, care advice from Chetan his team has faultless. Highly recommended." A.S. "Yet again I blown away by Cliniq. After my filling fell out Sunday night, was distraught. But phoned in got appointment same morning. Dr Kaher put me at ease, even though very anxious, did amazing job my..." Robyn B "What great dental clinic having issues with tooth 

In [37]:
def ai_runner(doc_list,caution_word_list=[],enable_db=False):
    doc_list_ccc = []
    for corrected_doc in grammar_check_main_concurrent(doc_list):
        print('Working on ' + corrected_doc.get('title',''))
        temp_doc = {}
        temp_doc = get_corrected_highlights(corrected_doc.get('chunks_corrected'))
        corrected_doc['main_data'] = temp_doc
        corrected_doc['status'] = 'done'
        del corrected_doc['chunks_corrected']
        doc_list_ccc.append(corrected_doc)

    if enable_db:
        status = update_processed_documents(doc_list_ccc)
        print('Database updated!',status)
    return doc_list_ccc

In [38]:
res = ai_runner(reterived_documents,caution_word_list=["Best", "Specialist", "Specialised", "Finest"],enable_db=True)

Working on Medical Aesthetics
Processed document: Medical Aesthetics


/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:92: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  for sentence in doc.dict().get('sentences',[]):
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_lower'] = df[column_name].str.lower()
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = 

Working on HOME
Processed document: HOME


/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:92: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  for sentence in doc.dict().get('sentences',[]):
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_lower'] = df[column_name].str.lower()
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = 

Document with _id: 66b48ab7f8c0761c30ba8d1d updated successfully.
Document with _id: 66b48cee940cc27674a58747 updated successfully.
Database updated! Documents updated successfully.


In [52]:
def empty_database(db):
    collections = db.list_collection_names()
    for collection_name in collections:
        collection = db[collection_name]
        result = collection.delete_many({})
        print(f"Deleted {result.deleted_count} documents from collection '{collection_name}'")

In [78]:
db

Database(MongoClient(host=['digimax-shard-00-00.bh5xd.mongodb.net:27017', 'digimax-shard-00-02.bh5xd.mongodb.net:27017', 'digimax-shard-00-01.bh5xd.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='digimax', authsource='admin', replicaset='atlas-14dxbq-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x12dff3fa0>), 'your_database_name')

In [79]:
empty_database(db)

Deleted 4 documents from collection 'processed_pages'
Deleted 1 documents from collection 'parent_docs'


In [56]:
def update_parent_doc_status_by_url(url, new_status):
    query = {'url': url}
    update = {'$set': {'status': new_status}}
    
    result = parent_docs_collection.update_one(query, update)
    
    if result.matched_count == 0:
        return f"No document found with URL: {url}"
    elif result.modified_count == 0:
        return f"Document with URL: {url} was already in the desired status: {new_status}"
    else:
        return f"Document with URL: {url} updated successfully to status: {new_status}"

In [64]:
url = "https://www.smilecliniq.com"
document_list = reterive_split_text(url=url,mode="crawl",max_pages=3,enable_db=True)


Crawled
Website added! UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000034'), 'opTime': {'ts': Timestamp(1723111501, 8), 't': 52}, 'upserted': ObjectId('66b4984d30ed5293cf6f7978'), 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1723111501, 8), 'signature': {'hash': b'3\xc0\xd2\xf0\xa7Cj\x84k\xa7\xb0F[y$\x96\x1d\x8a\x88p', 'keyId': 7353692852154531846}}, 'operationTime': Timestamp(1723111501, 8), 'updatedExisting': False}, acknowledged=True)
InsertManyResult([ObjectId('66b4984d940cc27674a58750'), ObjectId('66b4984d940cc27674a58751'), ObjectId('66b4984d940cc27674a58752')], acknowledged=True)


In [68]:
reterived_documents = get_first_two_init_pages_with_parent(url)

In [69]:
reterived_documents

[{'_id': ObjectId('66b4984d940cc27674a58750'),
  'url': 'https://www.smilecliniq.com/',
  'title': 'HOME',
  'page_content': 'OPEN EVENINGS AND WEEKENDS St John\'s Wood Finchley Dentist Close Dental Care Orthodontics Implants Specialist Services Laser Dentistry Other 1. 2. Half price examination now 95fees.php 0 APR Available Calculatorinterestfreedentaltreatment.php "My wife needed to see a dentist on an emergency basis Saturday after bad fall the evening before. We called morning and they were able take us that afternoon. The quality of care, both technically interpersonally,..." Hal N. "I\'ve been going Smile Cliniq for past couple years, treatment, care advice from Chetan his team has faultless. Highly recommended." A.S. "Yet again I have blown away by Cliniq. After my filling fell out Sunday night, was distraught. But phoned in got appointment same morning. Dr Kaher put me at ease, even though ve..." Robyn B "What great dental clinic having issues with tooth Sarita Kotecha brillia

In [70]:
while True:
    reterived_documents = get_first_two_init_pages_with_parent(url)
    print('Length of Remaining docs',len(reterived_documents))
    if len(reterived_documents) == 0:
        print('All Document Completed')
        status = update_parent_doc_status_by_url(url, 'done')
        print(status)
        break
    res = ai_runner(reterived_documents,caution_word_list=["Best", "Specialist", "Specialised", "Finest"],enable_db=True)
    print(res)

Length of Remaining docs 3
Working on TEAM
Processed document: TEAM


/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:92: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  for sentence in doc.dict().get('sentences',[]):
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_lower'] = df[column_name].str.lower()
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = 

Working on Training - Dr Chetan Kaher
Processed document: Training - Dr Chetan Kaher


/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:92: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  for sentence in doc.dict().get('sentences',[]):
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


Working on HOME
Processed document: HOME
Document with _id: 66b4984d940cc27674a58752 updated successfully.
Document with _id: 66b4984d940cc27674a58751 updated successfully.


/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:92: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  for sentence in doc.dict().get('sentences',[]):
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_lower'] = df[column_name].str.lower()
/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_11103/2563072748.py:78: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = 

Document with _id: 66b4984d940cc27674a58750 updated successfully.
Database updated! Documents updated successfully.
[{'_id': ObjectId('66b4984d940cc27674a58752'), 'url': 'https://www.smilecliniq.com/team.php', 'title': 'TEAM', 'page_content': 'OPEN EVENINGS AND WEEKENDS St John\'s Wood Finchley Dentist Close Dental Care Orthodontics Implants Specialist Services Laser Dentistry Other TEAM Dentists Principal Associate With Special Interest In Endodotics " Implant Surgeon Oral Surgery Anaesthetists Consultant in Sedation and Specialists Periodontist Orthodontist Ceramist CALL US ON 020 3199 4519 OR Call for a consultation today 4518 Our General Cosmetic Gum Treatments Copyright Smile Cliniq 2024 John Site Last Updated 30 July Limited. A company registered England Wales. Company number 08279068. Registered office CO TG Associates 7 Jardine House, Harrovian Business Village, Bessborough Road, Harrow, Middlesex, HA1 3EX Website by We use cookies on our website. Some of them are essential, wh

In [71]:
reterived_documents = get_first_two_init_pages_with_parent(url)

In [72]:
reterived_documents

[]

In [80]:
read_processed_pages({})

[]

In [81]:
read_parent_docs({})

[{'_id': ObjectId('66b4aef830ed5293cf7f1a10'),
  'url': 'https://example.com/',
  'halt_status': False,
  'status': 'done',
  'timestamp': datetime.datetime(2024, 8, 8, 16, 42, 55, 383000)},
 {'_id': ObjectId('66b4aef930ed5293cf7f1ac0'),
  'url': 'https://httpbin.org/get',
  'halt_status': False,
  'status': 'done',
  'timestamp': datetime.datetime(2024, 8, 8, 16, 41, 42, 709000)},
 {'_id': ObjectId('66b4aefa30ed5293cf7f1ba8'),
  'url': 'https://jsonplaceholder.typicode.com/posts',
  'halt_status': False,
  'status': 'done',
  'timestamp': datetime.datetime(2024, 8, 8, 16, 41, 42, 709000)},
 {'_id': ObjectId('66b4af9930ed5293cf7f81cf'),
  'url': 'https://example1.com/',
  'halt_status': False,
  'status': 'done',
  'timestamp': datetime.datetime(2024, 8, 8, 16, 44, 8, 403000)}]